# Stitching multiple images

This notebook is used to find a solution to the multi-graph synchronization problem and stitch images together.

In [ ]:
import cv2 as cv
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import ipynb.fs.defs.Utils as Utils
import ipynb.fs.defs.ImageStitcher as ImageStitcher

In [ ]:
#This function allows to perform multi-graph-synchronization stitching
def multi_graph_stitching(dataset_name,  #Name of the dataset to be used
                            imgs, #Images to be stitched
                            T_norm, #Normalization matrix
                            M, #Matrix containing the homographies between the images
                            C, #Replicas constraint matrix
                            imgs_translations = None, #Translation matrices to apply initially to the images
                            idx_ref = 0, #Index of the reference image
                            idxs = None, #Indexes of the subset of images to be stitched
                            verbose = True, #If True allows to print intermediate results
                            save_output = True, #If True allows to save the output
                            beautify = True, #If True allows to print the stitched image in a better way
                            stitching_dir = "stitched", #Directory where to save the results
                            graph_stitching_dir = "multi_graph_stitching", #Directory where to save the obtained graph
                            warp_shape = [10000,10000] ): #Shape of the stitched image
    
    #Output directory
    output_dir = os.path.join(os.path.join(stitching_dir,dataset_name),graph_stitching_dir)
    #Check whether the directory exists and create it
    if save_output:
        if os.path.isdir(output_dir):   
            shutil.rmtree(output_dir)
        if not os.path.isdir(output_dir):   
            os.makedirs(output_dir)
          
    
    #Compute matrix S
    S = (np.eye(M.shape[0])-C@np.linalg.pinv(C))@M.transpose()@M
    
    #Get eigenvalues and eigenvectors from S
    w,v = np.linalg.eig(S)
    
    #Remove imaginary parts
    w = np.real(w)
    v = np.real(v)
    
    #Number of singular vectors to be considered
    d = 3
    
    #Compute the offset for singular vectors
    if C.size == 0: # No multi arc case
        off = 0
    else:
        off = np.linalg.matrix_rank(C)
    
    #Get the indexes of eigenvalues
    indx = np.argsort(w)
    indxs = np.array([indx]*indx.shape[0])
    
    #Retrieve desired eigenvalues and eigenvectors
    w_sort = w[indx]
    v_sort = np.take_along_axis(v, indxs, axis=1)
    
    #Find the three right singular vectors associated with the 3 smallest singular values (so the last 3 columns)
    u_hat = v_sort[:,off:d+off]
    
    #Get the state of each node
    U = Utils.split_states(u_hat)[:len(imgs)] 

    #Retrieve the homographies w.r.t. the reference image
    H = Utils.get_homographies_from_states(U, idx_ref)
    
    
    #Stitch images together
    stitcher = ImageStitcher.ImageStitcher(imgs)
    Ht, stitch = stitcher.stitch_images(H, idx_ref, T_norm, idxs =idxs, imgs_translations = imgs_translations, beautify=beautify, size = warp_shape) #Stitch all the images
    
    #Save and print output if required
    if save_output:
        cv.imwrite(os.path.join(output_dir,"stitched.jpg"), cv.cvtColor(stitch,cv.COLOR_RGB2BGR))
    
    if verbose:
        figure(figsize=(40, 40), dpi=80)
        plt.imshow(stitch,),plt.show() 

    return H, stitch, Ht